In [1]:
my_ProjectID = "medical-imaging-ai"

In [2]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  medical-imaging-ai


In [3]:
import numpy as np
import pandas as pd


In [19]:
# CT image sample
#my_SeriesInstanceUID = "1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095"
my_StudyInstanceUID = "1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095"
collection_ID = "lctsc"
# pathology image sample
#my_SeriesInstanceUID = "1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0"

In [ ]:
%%bigquery idc_summary_df --project=$my_ProjectID 

SELECT
  COUNT(DISTINCT(PatientID)) AS total_patients,
  COUNT(DISTINCT(StudyInstanceUID)) AS total_studies,
  COUNT(DISTINCT(SeriesInstanceUID)) AS total_series,
  COUNT(DISTINCT(SOPInstanceUID)) AS total_instances,
  COUNT(DISTINCT(collection_id)) AS total_collections,
  SUM(instance_size)/(POW(1024,4)) AS total_size_TB
FROM
  `bigquery-public-data.idc_current.dicom_all`

In [ ]:
idc_summary_df

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = f"\
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` \
  WHERE \
    StudyInstanceUID = \"{my_StudyInstanceUID}\""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [23]:
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = f"\
  WITH studies AS (SELECT DISTINCT StudyInstanceUID FROM `bigquery-public-data.idc_current.dicom_all` \
                  WHERE collection_id =  \"{collection_ID}\"  \
                    LIMIT 10) \
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` INNER JOIN studies USING(StudyInstanceUID) \
    "

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [20]:
collection_ID

'lctsc'

In [21]:
selection_query = f"\
  SELECT DISTINCT StudyInstanceUID FROM `bigquery-public-data.idc_current.dicom_all` \
        WHERE collection_id =  \"{collection_ID}\"  \
         LIMIT 10"

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [24]:
pd.set_option('display.max_colwidth', None)

selection_df

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,1.3.6.1.4.1.14519.5.2.1.7014.4598.209403017805510796875124464511,1.3.6.1.4.1.14519.5.2.1.7014.4598.170975846739654511307430051907,1.3.6.1.4.1.14519.5.2.1.7014.4598.257972176703163406058162151287,gs://public-datasets-idc/ca1ba1d5-5c1c-4f73-bb26-e4db058f585c.dcm
1,1.3.6.1.4.1.14519.5.2.1.7014.4598.196761816459686710497287530585,1.3.6.1.4.1.14519.5.2.1.7014.4598.317364588283152286043395670052,1.3.6.1.4.1.14519.5.2.1.7014.4598.117069377656920418822706349404,gs://public-datasets-idc/2596d38c-b7e2-4b9e-ac8f-8f9d7f11f294.dcm
2,1.3.6.1.4.1.14519.5.2.1.7014.4598.120574963399566731807259976793,1.3.6.1.4.1.14519.5.2.1.7014.4598.307467503617501751325674523800,1.3.6.1.4.1.14519.5.2.1.7014.4598.118154082839637265883062898820,gs://public-datasets-idc/76e25014-c83b-42cd-b6d5-606ae348acbf.dcm
3,1.3.6.1.4.1.14519.5.2.1.7014.4598.120574963399566731807259976793,1.3.6.1.4.1.14519.5.2.1.7014.4598.307467503617501751325674523800,1.3.6.1.4.1.14519.5.2.1.7014.4598.334243897308450184324859382504,gs://public-datasets-idc/319d09af-5613-418a-b450-56d096f69638.dcm
4,1.3.6.1.4.1.14519.5.2.1.7014.4598.225420722952260716557532766670,1.3.6.1.4.1.14519.5.2.1.7014.4598.186463975978084836948875020803,1.3.6.1.4.1.14519.5.2.1.7014.4598.144061402316640628811935326508,gs://public-datasets-idc/465afd96-f71e-4ed5-a893-ceb8daf2c36b.dcm
...,...,...,...,...
1523,1.3.6.1.4.1.14519.5.2.1.7014.4598.196761816459686710497287530585,1.3.6.1.4.1.14519.5.2.1.7014.4598.317364588283152286043395670052,1.3.6.1.4.1.14519.5.2.1.7014.4598.235553504054372848260518454212,gs://public-datasets-idc/88b6d565-aa9a-4575-bfc4-0b5456168cfc.dcm
1524,1.3.6.1.4.1.14519.5.2.1.7014.4598.196761816459686710497287530585,1.3.6.1.4.1.14519.5.2.1.7014.4598.317364588283152286043395670052,1.3.6.1.4.1.14519.5.2.1.7014.4598.114606482958224558652604178975,gs://public-datasets-idc/2cdba64d-45d9-4c1d-9919-e6056daeac6b.dcm
1525,1.3.6.1.4.1.14519.5.2.1.7014.4598.387640041506394166677046351855,1.3.6.1.4.1.14519.5.2.1.7014.4598.143348938843553837741978176339,1.3.6.1.4.1.14519.5.2.1.7014.4598.133072353425264463188851098689,gs://public-datasets-idc/1aa09737-734d-490a-8d04-1bfd6729c888.dcm
1526,1.3.6.1.4.1.14519.5.2.1.7014.4598.209403017805510796875124464511,1.3.6.1.4.1.14519.5.2.1.7014.4598.170975846739654511307430051907,1.3.6.1.4.1.14519.5.2.1.7014.4598.261035465518407496575985737232,gs://public-datasets-idc/c764094c-bd2c-4f60-98be-d7573362b44a.dcm


In [25]:
# save the list of files
# save the list of GCS URLs into a file
import os
idc_download_folder = "./IDC_downloads"
if not os.path.exists(idc_download_folder):
  os.mkdir(idc_download_folder)

selection_manifest = os.path.join(idc_download_folder, "lctsc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [27]:
# copy files to staging area
# download is this simple!
!cat ./IDC_downloads/lctsc_manifest.txt | gsutil -m cp -I gs://miai-staging_bucket/LCTSC

Copying gs://public-datasets-idc/ca1ba1d5-5c1c-4f73-bb26-e4db058f585c.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/2596d38c-b7e2-4b9e-ac8f-8f9d7f11f294.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/76e25014-c83b-42cd-b6d5-606ae348acbf.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/319d09af-5613-418a-b450-56d096f69638.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/465afd96-f71e-4ed5-a893-ceb8daf2c36b.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/36cc8974-969a-44f4-b97d-17a41d069b4a.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/471af15a-a8a1-4881-b3e4-54a3bf744dbf.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/24f89ae7-9323-42a9-b755-7ff18d5296a8.dcm [Content-Type=application/dicom]...
Copying gs://public-datasets-idc/29f478f6-502c-4462-8473-eeca0fd77a91.dcm [Content-Type=application/dicom]...
Copying gs

In [ ]:
!gsutil ls gs://miai-staging_bucket/CPTAC-LSCC